<a href="https://colab.research.google.com/github/Yangyeeun/20MachineLearning/blob/master/%EC%BC%80%EB%9D%BC%EC%8A%A4%EC%99%80_%ED%85%90%EC%84%9C%ED%94%8C%EB%A1%9C_%ED%97%88%EB%B8%8C%EB%A5%BC_%EC%82%AC%EC%9A%A9%ED%95%9C_%EC%98%81%ED%99%94_%EB%A6%AC%EB%B7%B0_%ED%85%8D%EC%8A%A4%ED%8A%B8_%EB%B6%84%EB%A5%98%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 케라스와 텐서플로 허브를 사용한 영화 리뷰 텍스트 분류하기

이진분류 Binary Classification 예시로 
이전까지 이미지분류였다면 영화리뷰 텍스트를 긍정,부정으로 분류해보자!

 IMDB 데이터셋:Internet Movie Database에서 수집한 50,000개의 영화 리뷰 텍스트 
 = 25,000개  훈련용+ 25,000개 리뷰 테스트용

케라스: 텐서플로의 모델생성, 훈련을 위한 고수준의 파이썬 API

텐서플로 허브: 전이 학습 라이브러리

In [ ]:
import numpy as np

import tensorflow as tf

!pip install tensorflow-hub
!pip install tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("버전: ", tf.__version__)
print("즉시 실행 모드: ", tf.executing_eagerly())
print("허브 버전: ", hub.__version__)
print("GPU", "사용 가능" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

## 1.IMDB 데이터셋 다운로드

In [ ]:
# 훈련 세트= 6대 4로 나눕니다.
# 전체 50000개 = 훈련 15,000개+ 검증 10,000개 + 테스트 25,000개 샘플
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

## 데이터 탐색
데이터 형태: 전처리된 정수 배열-> 영화 리뷰에 나오는 단어 표현
레이블: 0은 부정적인 리뷰, 1은 긍정적인 리뷰

In [ ]:
#처음 10개의 샘플,레이블을 출력
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

train_labels_batch
 #<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

## 모델 구성

신경망의 층을 구성할때 생각해야할 것

*   어떻게 텍스트를 표현할 것인가?
*   모델에서 얼마나 많은 층을 사용할 것인가?
*   각 층에서 얼마나 많은 hidden unit을 사용할 것인가?

*   어떻게 텍스트를 표현할 것인가?
 - 입력: 문장/ 출력: 0,1 레이블
 -> 텍스트 표현 = embedding 벡터로 변환
 -> 첫 번째 층으로 사전 훈련된 텍스트 임베딩을 사용 (사람이 쓰는 자연어를 기계가 이해할 수 있는 숫자형태인 vector로 바꾼 결과 )
 => 장점1:텍스트 전처리에 대해 신경X


In [ ]:
#텐서플로 허브 모델을 사용하는 케라스 층 생성, 샘플로 테스트
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])